<a href="https://colab.research.google.com/github/inamansari21/datascience/blob/main/random_forest_new_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import datasets,tree
from sklearn.tree import export_graphviz 
from sklearn import externals
from io import StringIO
import pydotplus
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv("Fraud_check.csv")


In [ ]:
df1 = df.copy()


In [ ]:
df1.head()

In [ ]:
df1.describe()


In [ ]:
df1.isnull().sum()


In [ ]:
df1.dtypes


# **checking outlier**

In [ ]:
ax = sns.boxplot(df1['Taxable.Income'])


there are no outliers in the data

In [ ]:
plt.rcParams["figure.figsize"] = 9,5


In [ ]:
plt.figure(figsize=(16,5))
print("Skew: {}".format(df1['Taxable.Income'].skew()))
print("Kurtosis: {}".format(df1['Taxable.Income'].kurtosis()))
ax = sns.kdeplot(df1['Taxable.Income'],shade=True,color='g')
plt.xticks([i for i in range(10000,100000,10000)])
plt.show()

The data is Skwed on the right.

The data has negative Kurtosis


In [ ]:
obj_colum = df1.select_dtypes(include='object').columns.tolist()


In [ ]:
plt.figure(figsize=(16,10))
for i,col in enumerate(obj_colum,1):
    plt.subplot(2,2,i)
    sns.countplot(data=df1,y=col)
    plt.subplot(2,2,i+1)
    df1[col].value_counts(normalize=True).plot.bar()
    plt.ylabel(col)
    plt.xlabel('% distribution per category')
plt.tight_layout()
plt.show()  


In [ ]:
num_columns = df1.select_dtypes(exclude='object').columns.tolist()


In [ ]:
plt.figure(figsize=(18,40))
for i,col in enumerate(num_columns,1):
    plt.subplot(8,4,i)
    sns.kdeplot(df[col],color='g',shade=True)
    plt.subplot(8,4,i+10)
    df[col].plot.box()
plt.tight_layout() 
plt.show()
num_data = df[num_columns]
pd.DataFrame(data=[num_data.skew(),num_data.kurtosis()],index=['skewness','kurtosis'])


In [ ]:
num_columns = df1.select_dtypes(exclude='object').columns.tolist()


In [ ]:
plt.figure(figsize=(18,40))
for i,col in enumerate(num_columns,1):
    plt.subplot(8,4,i)
    sns.kdeplot(df[col],color='g',shade=True)
    plt.subplot(8,4,i+10)
    df[col].plot.box()
plt.tight_layout() 
plt.show()
num_data = df[num_columns]
pd.DataFrame(data=[num_data.skew(),num_data.kurtosis()],index=['skewness','kurtosis'])


In [ ]:
df1 = pd.get_dummies(df1, columns = ['Undergrad','Marital.Status','Urban'])


In [ ]:
corr = df1.corr()


In [ ]:
corr = df1.corr()


In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(corr,annot=True)


## Random Forest Model
Since the target variable is continious, we create a class of taxable_income <= 30000 as "Risky" and others are "Good"


In [ ]:
df1['Taxable.Income']=pd.cut(df1['Taxable.Income'],bins=[0,30000,100000],labels=['risky','good'])


In [ ]:
list(df1.columns)


In [ ]:
X = df1.iloc[:,1:10]
y = df1.iloc[:,0]


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)


In [ ]:
y_train.value_counts()


In [ ]:
model =RF(n_jobs=4,n_estimators = 150, oob_score =True,criterion ='entropy') 
model.fit(x_train,y_train)
model.oob_score_


In [ ]:
pred_train = model.predict(x_train)


In [ ]:
accuracy_score(y_train,pred_train)


In [ ]:
confusion_matrix(y_train,pred_train)


In [ ]:
pred_test = model.predict(x_test)


In [ ]:
accuracy_score(y_test,pred_test)


In [ ]:
confusion_matrix(y_test,pred_test)


In [ ]:
df_t=pd.DataFrame({'Actual':y_test, 'Predicted':pred_test})


In [ ]:
df_t


In [ ]:
cols = list(df1.columns)


In [ ]:
predictors = cols[1:10]
target = cols[0]


In [ ]:
tree1 = model.estimators_[20]


In [ ]:
dot_data = StringIO()


In [ ]:
export_graphviz(tree1, out_file = dot_data, feature_names =predictors, class_names = target, filled =True,rounded=True,impurity =False,proportion=False,precision =2)


In [ ]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())


In [ ]:
graph.write_png('fraud_full.png')


## Conclusion
Since the accuracy of the Training set is 100%.
 we test the accurancy on the test data which is 72%

As seen in the confusion matrix of Test data 94 instances are presdected correctly and 26 instances are not


In [ ]:
rf_small = RF(n_estimators=10, max_depth = 3)


In [ ]:
rf_small.fit(x_train,y_train)


In [ ]:
tree_small = rf_small.estimators_[5]


In [ ]:
export_graphviz(tree_small, out_file = dot_data, feature_names = predictors, rounded = True, precision = 1)


In [ ]:
graph_small = pydotplus.graph_from_dot_data(dot_data.getvalue())


In [ ]:
graph.write_png('fraud_small.png')


In [ ]:
img = mpimg.imread('fraud_small.png') 


In [ ]:
plt.imshow(img)


In [ ]:
model.feature_importances_


In [ ]:
fi = pd.DataFrame({'feature': list(x_train.columns),
                   'importance': model.feature_importances_}).\
                    sort_values('importance', ascending = False)


In [ ]:
fi

As seen in the above table city population is most important feature.
